In [ ]:
from google.cloud import bigquery

# --- CONFIGURATION ---
project_id = "qwiklabs-gcp-00-711e7fbdb4c8"
dataset_id = "AuroraFAQs"
location = "US"

# --- INITIALIZE CLIENT ---
client = bigquery.Client(project=project_id)

# --- DEFINE DATASET REFERENCE ---
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
dataset_ref.location = location

# --- CREATE DATASET IF NOT EXISTS ---
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset '{dataset_id}' already exists.")
except Exception:
    client.create_dataset(dataset_ref)
    print(f"Dataset '{dataset_id}' created in location '{location}'.")

In [ ]:
# --- CREATE TABLE ---
sql_create = """

CREATE TABLE IF NOT EXISTS AuroraFAQs.aurora_faqs (
    question STRING,
    answer STRING
);
"""

# --- RUN CREATE TABLE JOB ---
query_job = client.query(sql_create)

In [ ]:
# --- LOAD FILE ---
sql_load = """
LOAD DATA OVERWRITE AuroraFAQs.aurora_faqs
(question STRING,
    answer STRING)
FROM FILES (
    format = 'CSV',
    skip_leading_rows=1,
    uris = ['gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv']
);
"""

# --- RUN LOAD FILE JOB ---
query_job = client.query(sql_load)

In [ ]:
'''
    Set up external Connection us.embedding_conn connecting to Vertex AI: BigQuery Federation Connection Type
      Vertex AI remote models, remote functions and BigLake (Cloud Resource)
    Provided needed permissions to Service Account id via IAM tab
    All done via GCP Interface
'''

In [ ]:
# --- LOAD MODEL ---
sql_load_embeddings = """
CREATE OR REPLACE MODEL AuroraFAQs.Embeddings
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""

# --- RUN LOAD MODEL JOB ---
query_job = client.query(sql_load_embeddings)

In [ ]:
# --- CREATE EMBEDDINGS ---
sql_create_embeddings = """
CREATE OR REPLACE TABLE AuroraFAQs.aurora_faqs_embedded AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `AuroraFAQs.Embeddings`,
    (SELECT CONCAT(question, ': ', answer) AS content FROM `AuroraFAQs.aurora_faqs`)
);
"""

# --- RUN CREATE EMBEDDINGS JOB ---
query_job = client.query(sql_create_embeddings)


In [ ]:
# --- QUERY ---
question = "local" # Input question to be searched against the vector results

sql_query_embeddings = """
CREATE OR REPLACE TABLE `AuroraFAQs.vector_search_result` AS
SELECT
    query.query,
    base.content
FROM
    VECTOR_SEARCH(
        TABLE `AuroraFAQs.aurora_faqs_embedded`,
        'ml_generate_embedding_result',
        (
            SELECT
                ml_generate_embedding_result,
                content AS query
            FROM
                ML.GENERATE_EMBEDDING(
                    MODEL `AuroraFAQs.Embeddings`,
                    (SELECT '""" + question + """' AS content)
                )
        ),
        top_k => 5,
        options => '{"fraction_lists_to_search": 0.01}'
    );
"""

# --- RUN QUERY JOB ---
query_job = client.query(sql_query_embeddings)


In [ ]:
# --- LOAD GEMINI ---
sql_load_gemini = """
CREATE OR REPLACE MODEL AuroraFAQs.Gemini
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');
"""

# --- RUN LOAD MODEL JOB ---
query_job = client.query(sql_load_gemini)

In [ ]:
# --- CREATE QUERY FUNCTION ---

def search_similar_items(question: str):
    # Safely escape single quotes
    question_safe = question.replace("'", "\\'")

    query = f"""
    SELECT
      result.base.content AS content,
      result.query.content AS query,
      result.distance AS distance
    FROM
      VECTOR_SEARCH(
        TABLE `AuroraFAQs.aurora_faqs_embedded`,
        'ml_generate_embedding_result',
        (
          SELECT
            content,
            ml_generate_embedding_result
          FROM
            ML.GENERATE_EMBEDDING(
              MODEL `AuroraFAQs.Embeddings`,
              (SELECT '{question_safe}' AS content)
            )
        )
        , top_k => 5,
        options => '{{"fraction_lists_to_search": 0.01}}'
      ) AS result
    """

    query_job = client.query(query)
    results = query_job.result()

    # Return content, original query, and similarity distance
    return [(row.content, row.query, row.distance) for row in results]


# --- HAVE GEMINI SUMMARIZE ---

def generate_answer_with_gemini(user_question: str):
    # Step 1: Run vector search
    top_matches = search_similar_items(user_question)

    if not top_matches:
        return "Sorry, I couldn't find any relevant information."

    # Step 2: Build prompt
    reviews_text = ",\n".join([f"review text: {text}" for text in top_matches])
    full_prompt = f"Summarize retrieved FAQ results from customer question: {user_question}\n{reviews_text}"

    # Step 3: Prepare safe parameterized query
    query = """
    SELECT
        ml_generate_text_llm_result AS generated
    FROM
        ML.GENERATE_TEXT(
            MODEL `AuroraFAQs.Gemini`,
            (
                SELECT @prompt AS prompt
            ),
            STRUCT(
                0.4 AS temperature,
                300 AS max_output_tokens,
                0.5 AS top_p,
                5 AS top_k,
                TRUE AS flatten_json_output
            )
        )
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("prompt", "STRING", full_prompt)
        ]
    )

    results = client.query(query, job_config=job_config).result()

    for row in results:
        return row.generated


# ---- CHAT LOOP ----
def chat():
    print("Please ask me a question (Type 'exit' to quit)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ("exit", "quit"):
            print("Goodbye")
            break

        try:
            # Generate Gemini answer using top matching FAQ content
            response = generate_answer_with_gemini(user_input)
            print(f"\nGemini: {response}\n")
        except Exception as e:
            print(f"Error: {e}")



# ---- RUN CHAT ----
if __name__ == "__main__":
    chat()